In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skcriteria import Data, MIN, MAX
from skcriteria.madm import closeness, simple

## Bring in data

In [ ]:
lst_path = './aqua95_mtCounties.geojson' 
census_path = '../../../../Data/Census/acs2017_county_data_MT.csv'
res_path = '../shapefiles/MontanaReservations_shp/MontanReservations_4326.shp'
kh_path = './killer-heat-data-by-county-100.csv'
kh_cols = ['State', 'County', 'Historical', 'MC_slow', 'MC_no', 'EC_slow', 'EC_no', 'Rapid_Action']
lst_gdf = gpd.read_file(lst_path)
cen_df = pd.read_csv(census_path)
res_bounds = gpd.read_file(res_path)
kh_df = pd.read_csv(kh_path, skiprows=3, names=kh_cols)
kh_df = kh_df[kh_df['State'] == 'MT']
kh_df.head()

## Prepare DataFrame for analysis

In [ ]:
lst_gdf['mean_F'] = (((lst_gdf['mean'] * 0.02) - 273.15) * (9/5.)) + 32
lst_gdf['GEOID'] = lst_gdf['GEOID'].astype(int)
kh_df['killer_heat_diff'] = kh_df['MC_no'] - kh_df['Historical']
cen_df = pd.merge(cen_df, kh_df[['County', 'killer_heat_diff']], how='inner', left_on='County', right_on='County')
gdf = pd.merge(lst_gdf, cen_df, how='inner', left_on='GEOID', right_on='CountyId')
cols = ['CountyId', 'mean_F', 'killer_heat_diff', 'Income', 'Poverty', 'ChildPoverty', 'Construction', 'Production', 'Unemployment', 'Professional']
data = gdf[cols]
data = data.set_index('CountyId')
data_norm = data/((data**2).sum())**0.5

## Randomly weighted MCA

In [ ]:
crit = [MIN, MIN, MAX, MIN, MIN, MIN, MIN, MIN, MAX]
counties = data.index
cnms = data.columns
mtx = data.values

n = 10000
res = np.zeros([len(data), n])
for i in range(n):
    rand_wts = np.random.dirichlet(np.ones(7), size=1)*.49
    wts = list(np.insert(rand_wts, 0, [0.255, 0.255]))
    mca_data = Data(mtx, crit, anames=counties, cnames=cnms, weights=wts)
    dm = closeness.TOPSIS()
    dec = dm.decide(mca_data)
    #res[:, i] = dec.rank_
    res[:, i] = dec.e_.closeness
# res_df = pd.DataFrame({
#         "name": gdf['NAME'], 
#         "min_rank": res.min(axis=1),
#         "max_rank": res.max(axis=1),
#         "median_rank": np.median(res, axis=1).round(0)
#         })
res_df = pd.DataFrame(res)
res_df.index = gdf['NAME'] 
res_df = res_df.T

## Save results as csv

In [ ]:
wts

In [ ]:

# res_df.to_csv('prelim_county_vulnerability_rank_uncertainty.csv')

## Plot w/ Seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
res_melt = pd.melt(res_df)
plt.figure(figsize=(6,10))
sns.set(font_scale=.9)
ax = sns.boxplot(data=res_melt, x='value', y='NAME', fliersize=1)
ax.set(ylabel='County', xlabel='Closeness')

In [ ]:
fig = ax.get_figure()
fig.savefig("ranked_heat_vulnerability_uncertainty_closeness.png", bbox_inches="tight", dpi=300)